In [7]:
from glob import glob
from model import Model
from utils import Colors, get_image_tensor, xywh2xyxy, xywh2xyxy, box_iou, ap_per_class
from PIL import Image
import cv2
import numpy as np

images = glob('/data/images/val/*')
labels = glob('/data/labels/val/*')

nc = 1

stats = []

iouv = np.linspace(0.5, 0.95, 10)  # iou vector for mAP@0.5:0.95
niou = iouv.size

In [8]:
def process_batch(detections, labels, iouv):
    """
    Return correct predictions matrix. Both sets of boxes are in (x1, y1, x2, y2) format.
    Arguments:
        detections (Array[N, 6]), x1, y1, x2, y2, conf, class
        labels (Array[M, 5]), class, x1, y1, x2, y2
    Returns:
        correct (Array[N, 10]), for 10 IoU levels
    """
    correct = np.zeros((detections.shape[0], iouv.shape[0])).astype(bool)
    iou = box_iou(labels[:, 1:], detections[:, :4])
    correct_class = labels[:, 0:1] == detections[:, 5]
    for i in range(len(iouv)):
        x = np.where((iou >= iouv[i]) & correct_class)  # IoU > threshold and classes match
        if x[0].shape[0]:
            matches = np.cat((np.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detect, iou]
            if x[0].shape[0] > 1:
                matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
                # matches = matches[matches[:, 2].argsort()[::-1]]
                matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
            correct[matches[:, 1].astype(int), i] = True
    return np.array(correct, dtype=bool)

In [9]:
model = Model('models/320/s.tflite', 'data/tree.yaml', conf_thresh=0.25, iou_thresh=0.45)

INFO:EdgeTPUModel:Confidence threshold: 0.25
INFO:EdgeTPUModel:IOU threshold: 0.45
INFO:EdgeTPUModel:Loaded 1 classes
INFO:EdgeTPUModel:Successfully loaded /home/mendel/code/yolo-with-coral/models/320/s.tflite


In [10]:
for img in images:
    img, shapes = get_image_tensor(cv2.imread(img), 320)
    
    print(model.forward(img))

NameError: name 'xywh2xyxy' is not defined

In [ ]:
import numpy as np

y = []
for label in labels:
    with open(label, 'r') as f:
        lines = []
        for line in f.readlines():
            lines.append(list(map(float, line.strip().split())))
        if lines:
            lines = np.array(lines)
        else:
            lines = np.ndarray((0, 5))
        y.append(lines)

In [ ]:
x = [get_image_tensor(image, model.input_size[0]) for image in images]

In [ ]:
preds = []
labels = []
for (full_image, net_image, pad), (label) in zip(x, y):
    pred = model.forward(net_image)
    preds.append(pred)
    labels.append(label)
    print('pred',pred)
    print('label',label)
#     print(model.get_scaled_coords(pred[0][:,:4], full_image, pad))

In [ ]:
net_image.shape[:2], pred[0][:, :4], [2160, 3840], shapes